# Perfilamiento sobre algoritmo Simplex

### 1. Medición de tiempo

Características de la instancia que utilizamos para el perfilamiento

In [18]:
%%bash
lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
Address sizes:       36 bits physical, 48 bits virtual
CPU(s):              8
On-line CPU(s) list: 0-7
Thread(s) per core:  2
Core(s) per socket:  4
Socket(s):           1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               142
Model name:          Intel(R) Core(TM) i7-8550U CPU @ 1.80GHz
Stepping:            10
CPU MHz:             1992.000
CPU max MHz:         1992.0000
BogoMIPS:            3984.00
Virtualization:      VT-x
Hypervisor vendor:   Windows Subsystem for Linux
Virtualization type: container
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush dts acpi mmx fxsr sse sse2 ss ht tm pbe syscall nx pdpe1gb rdtscp lm pni pclmulqdq dtes64 monitor ds_cpl vmx est tm2 ssse3 fma cx16 xtpr pdcm pcid sse4_1 sse4_2 x2apic movbe popcnt tsc_deadline_timer aes xsave osxsave avx f16c rdrand lahf_lm abm 3dnowprefetch fsgs

In [19]:
%%bash
sudo lshw -C memory

sudo: a terminal is required to read the password; either use the -S option to read from standard input or configure an askpass helper


CalledProcessError: Command 'b'sudo lshw -C memory\n'' returned non-zero exit status 1.

In [20]:
import math
import time
import SimplexC
import numpy as np
#from pytest import approx
from scipy.optimize import linprog

**Módulo time**

Ejemplo Maximización para evualuar tiempo de ejecución

In [21]:
c = [3, 5]
b = [4, 12, 18]
A = [[1,  0],
    [0,  2],
    [3, 2]]

start_time = time.time()
problema = SimplexC.Simplex(c,A,b,problem='Max')
method_result, opt, status = problema.solve()
end_time = time.time()
secs = end_time-start_time
print("Simplex algorithm tomó",secs,"segundos" )

Optimization completed successfully !
Solution for x vector:
[2.0, 6.0]
Optimal value:
-36.0
Simplex algorithm tomó 0.002106189727783203 segundos


Con Scipy

In [22]:
c_scipu=[-3, -5]

start_time = time.time()
opt = linprog(c=c_scipu, A_ub=A, b_ub=b,
              method="simplex")
end_time = time.time()
secs = end_time-start_time
print("Scipy  tomó",secs,"segundos" )

Scipy  tomó 0.007745981216430664 segundos


Obs. En esta primera parte nos damos cuenta que el algoritmo implementado es un poco más rápido que el de Scipy.

**Comando Magic %time**

In [23]:
%time problema.solve()

Optimization completed successfully !
Solution for x vector:
[0, 0]
Optimal value:
0
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 884 µs


([0, 0], 0, 0)

**Cprofile**

Para poder visualizar en que secciones de código se tarda más

In [24]:
import cProfile

In [25]:
cprof = cProfile.Profile()
cprof.enable()
problema = SimplexC.Simplex(c,A,b,problem='Max')
method_result, opt, status = problema.solve()
cprof.disable()
cprof.print_stats(sort='cumtime')

Optimization completed successfully !
Solution for x vector:
[2.0, 6.0]
Optimal value:
-36.0
         425 function calls in 0.003 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        3    0.000    0.000    0.002    0.001 interactiveshell.py:3386(run_code)
        3    0.000    0.000    0.002    0.001 {built-in method builtins.exec}
        1    0.000    0.000    0.002    0.002 <ipython-input-25-9a4092ce7cfb>:4(<module>)
       16    0.000    0.000    0.001    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
       10    0.000    0.000    0.001    0.000 iostream.py:384(write)
        4    0.000    0.000    0.001    0.000 <__array_function__ internals>:2(solve)
        4    0.000    0.000    0.001    0.000 linalg.py:313(solve)
       11    0.000    0.000    0.000    0.000 iostream.py:195(schedule)
        1    0.000    0.000    0.000    0.000 index_tricks.py:317(__getitem__)
        6    0.00

In [26]:
import pstats

cprof.dump_stats("Simplex_stats_cython")

In [27]:
p_simplex_stats = pstats.Stats("Simplex_stats_cython")
print(p_simplex_stats.sort_stats("cumulative").print_stats(10))

Sat May  8 15:56:11 2021    Simplex_stats_cython

         425 function calls in 0.003 seconds

   Ordered by: cumulative time
   List reduced from 77 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        3    0.000    0.000    0.002    0.001 /home/jreyesgar93/.pyenv/versions/3.9.2/envs/itam/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3386(run_code)
        3    0.000    0.000    0.002    0.001 {built-in method builtins.exec}
        1    0.000    0.000    0.002    0.002 <ipython-input-25-9a4092ce7cfb>:4(<module>)
       16    0.000    0.000    0.001    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
       10    0.000    0.000    0.001    0.000 /home/jreyesgar93/.pyenv/versions/3.9.2/envs/itam/lib/python3.9/site-packages/ipykernel/iostream.py:384(write)
        4    0.000    0.000    0.001    0.000 <__array_function__ internals>:2(solve)
        4    0.000    0.000    0.001    0.000

In [28]:
print(p_simplex_stats.sort_stats("cumulative").print_stats("solve|module"))

Sat May  8 15:56:11 2021    Simplex_stats_cython

         425 function calls in 0.003 seconds

   Ordered by: cumulative time
   List reduced from 77 to 6 due to restriction <'solve|module'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.002    0.002 <ipython-input-25-9a4092ce7cfb>:4(<module>)
        4    0.000    0.000    0.001    0.000 <__array_function__ internals>:2(solve)
        4    0.000    0.000    0.001    0.000 /home/jreyesgar93/.pyenv/versions/3.9.2/envs/itam/lib/python3.9/site-packages/numpy/linalg/linalg.py:313(solve)
        1    0.000    0.000    0.000    0.000 <ipython-input-25-9a4092ce7cfb>:3(<module>)
        1    0.000    0.000    0.000    0.000 <ipython-input-25-9a4092ce7cfb>:5(<module>)
        4    0.000    0.000    0.000    0.000 /home/jreyesgar93/.pyenv/versions/3.9.2/envs/itam/lib/python3.9/site-packages/numpy/linalg/linalg.py:309(_solve_dispatcher)




In [29]:
#numero de llamadas a funciones primitivas
print(p_simplex_stats.prim_calls)

425


In [30]:
p_simplex_stats.strip_dirs().sort_stats("cumulative").print_callers(10)

   Ordered by: cumulative time
   List reduced from 77 to 10 due to restriction <10>

Function                                                                 was called by...
                                                                             ncalls  tottime  cumtime
interactiveshell.py:3386(run_code)                                       <- 
{built-in method builtins.exec}                                          <-       3    0.000    0.002  interactiveshell.py:3386(run_code)
<ipython-input-25-9a4092ce7cfb>:4(<module>)                              <-       1    0.000    0.002  {built-in method builtins.exec}
{built-in method numpy.core._multiarray_umath.implement_array_function}  <-       1    0.000    0.000  <__array_function__ internals>:2(concatenate)
                                                                                  1    0.000    0.000  <__array_function__ internals>:2(copy)
                                                                                 

In [31]:
p_simplex_stats.strip_dirs().sort_stats("cumulative").print_callees("print|dot|ndim|solve")

   Ordered by: cumulative time
   List reduced from 77 to 8 due to restriction <'print|dot|ndim|solve'>

Function                                 called...
                                             ncalls  tottime  cumtime
<__array_function__ internals>:2(solve)  ->       4    0.000    0.000  linalg.py:309(_solve_dispatcher)
                                                  4    0.000    0.001  {built-in method numpy.core._multiarray_umath.implement_array_function}
linalg.py:313(solve)                     ->       4    0.000    0.000  linalg.py:102(get_linalg_error_extobj)
                                                  8    0.000    0.000  linalg.py:107(_makearray)
                                                  4    0.000    0.000  linalg.py:112(isComplexType)
                                                  4    0.000    0.000  linalg.py:135(_commonType)
                                                  4    0.000    0.000  linalg.py:193(_assert_stacked_2d)
                 

Obs. Se observa que las partes del código que tienen un mayor número de llamadas son

* **print**- El cual se utiliza al final del algoritmo para mostrar los valores óptimos 
* **dot**(producto punto)
* **dim** y **size** que se ocupan al principio del algoritmo para saber la longuitud de las matrices a optimizar
* **solve**- Este es el que tiene mayor número de llamadas

**Lineprofiler**

Para saber línea por línea en que parte nuestro algoritmo se está tardano más en ejecutar

In [34]:
import line_profiler

problema = SimplexC.Simplex(c,A,b,problem='Max')
line_prof = line_profiler.LineProfiler()
print(line_prof(problema.solve)())

Optimization completed successfully !
Solution for x vector:
[2.0, 6.0]
Optimal value:
-36.0
([2.0, 6.0], -36.0, 0)


In [35]:
print(line_prof.print_stats())

Timer unit: 1e-06 s

Total time: 0 s

Could not find file SimplexC/__init__.pyx
Are you sure you are running this program from the same directory
that you ran the profiler from?
Continuing without the function's contents.


ValueError: max() arg is an empty sequence